In [ ]:
from uuid import uuid4

import cv2
from imageai.Detection import ObjectDetection

In [ ]:
detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("model/yolo.h5")
detector.loadModel()

In [ ]:
img_array = cv2.imread("media/images/4.jpg")

In [ ]:
custom = detector.CustomObjects(person=True)

detections = detector.detectObjectsFromImage(
    custom_objects=custom,
    input_image=img_array,
    # output_image_path="media/output_images/image_new.jpg",
    minimum_percentage_probability=30,
    input_type="array",
    output_type="array"
)

In [ ]:
detections

In [ ]:
len(detections)

In [ ]:
cv2.imwrite("media/output_images/image_new_array.jpg", detections[0])

In [ ]:
vid = cv2.VideoCapture("media/input_video/video_2.mp4")

while vid.isOpened():
    # Capture frame-by-frame
    ret, frame = vid.read()
    print(ret)

    if ret:
        cv2.imwrite(f"media/images_from_video/__{uuid4()}.jpg", frame)

    # while not ret:
    #     print("Can't receive frame. Retrying ...")
    #     vid.release()
    #     vid = cv2.VideoCapture("media/input_video/video_2.mp4")
    #     ret, frame = vid.read()

    else:
        continue

    # When everything done, release the video capture object
    vid.release()


In [ ]:
import time
import multiprocessing as mp
from utils import get_all_frames
from main_p import detect_person
import cv2

cap = cv2.VideoCapture("media/input_video/video_2.mp4")

success, all_frames = get_all_frames(cap)

print(f"Length of all frames: {len(all_frames)}")

if success:
    start_time = time.time()

    pool = mp.Pool(mp.cpu_count())

    result = pool.starmap(
        detect_person,
        [(img, count) for count, img in all_frames.items()]
    )

    pool.close()

    print(f"Length of result: {len(result)}")

    width, height = (
        int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    )
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    number_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if len(result) == number_of_frames:
        # Define the codec and create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'avc1')  # TODO: change this, this is not working
        out = cv2.VideoWriter()
        output_file_name = "media/output_video/parallel_detected_video_2.mp4"
        out.open(output_file_name, fourcc, fps, (width, height), True)

        all_images = [result[count] for count in range(1, number_of_frames + 1)]

        _ = [out.write(image) for image in all_images]

        out.release()

    # Release resources
    cap.release()

    print(f"Time taken: {time.time() - start_time}")